# Visualization

In this notebook we are going to visualize interesting results

In [361]:
import pandas as pd
import bokeh
from folium import plugins
import folium
import numpy as np
import math
import os

# Preprocess

In [129]:
total_events = pd.read_csv(os.path.join('./total_events_with_geo.csv'))
total_events.dropna(inplace = True)
total_events.drop(['Date','City'], axis = 1, inplace = True)

#lats = total_events.drop_duplicates(subset=['Venue'])['Latitude'].values
#lons = total_events.drop_duplicates(subset=['Venue'])['Longitude'].values
#mags = total_events['Venue'].value_counts()

## HeatMap
heatmap = folium.Map(location=[np.mean(lats), np.mean(lons)], zoom_start=8, tiles = 'cartodbdark_matter',
               min_zoom=8,min_lat=45,max_lat=48,min_lon=5,max_lon=11)
heatmap.add_children(plugins.HeatMap(zip(lats, lons), radius = 10,min_opacity=0.7,blur=15))
lats = total_events['Latitude'].values
lons = total_events['Longitude'].values

## DetailedMap
map_detailed = folium.Map(location=[np.mean(lats), np.mean(lons)], zoom_start=8, tiles = 'cartodbdark_matter',
               min_zoom=8,min_lat=45,max_lat=48,min_lon=5,max_lon=11)

df_grouped = total_events.groupby(['Venue'])
aggregation = {'Latitude':'mean',
               'Longitude':'mean',
               'Artist':'count'
              }
df_p = df_grouped.agg(aggregation)

#Marker creation
for i in range(len(df_p)):
    popuptest = df_p.ix[i].name+' - '+str(math.floor(df_p.ix[i].Artist))+' artists'
    folium.CircleMarker(location=[df_p.ix[i].Latitude, df_p.ix[i].Longitude], 
                        radius=3*df_p.ix[i].Artist,
                    popup=popuptest, color='#3186cc',
                    fill_color='#3186cc').add_to(map_detailed)




# Heat Map

This map shows the number of concerts. The more red, the higher number of concerts happenned. It allows to immediatley see the areas with a lot of musical events.

In [362]:
heatmap.save('heatmap.html')
heatmap

# Detailed Map
This map shows the number of concerts. The bigger the circle, the higher number of concerts happened. This map is interractive, we can click on the circle to obtain the name of the venue and the number of concerts.

In [363]:
map_detailed.save('detailed_map.html')
map_detailed